# Documentação

!pip install yfinance

### Artigo original:
https://towardsdatascience.com/how-to-create-a-fully-automated-ai-based-trading-system-with-python-708503c1a907

### Modelo ARIMA:
https://pt.wikipedia.org/wiki/ARIMA

### Uso do Auto-ARIMA:
https://towardsdatascience.com/time-series-forecasting-using-auto-arima-in-python-bb83e49210cd

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

# from statsmodels.tsa.arima.model import ARIMA

# import pyotp
# import robin_stocks as robinhood
# import alpaca_trade_api as alpaca

# import telegram
import sys
import os

import matplotlib.pyplot as plt 
plt.style.use('fivethirtyeight')

In [120]:
def get_finance_data(ticker, period='1y', interval='1d'):
    '''
    Função para capturar os dados dos ativos, acrescentar ".SA" no final do ticker para ativos 
    negociados na Bovespa, exemplo "PETR4.SA".
    Exemplo url base da API: https://query1.finance.yahoo.com/v7/finance/options/PETR4.SA?date=20201222
    Exemplo url scrape da API: https://finance.yahoo.com/quote/PETR4.SA
    
    Parameters
    ----------
    period: default '1y', periodos validos: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
    inteval: default '1d', intervalos validos: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1d, 5d, 1wk, 1mo, 3mo.
    '''
    
    google = yf.Ticker(ticker)
    
    df = google.history(period=period, interval=interval)
#     df['date'] = pd.to_datetime(df.index).time
#     df.set_index('date', inplace=True)

    return df


def get_forecast():
    df = get_finance_data()

    # Assuming that we've properly trained the model before and that the 
    # hyperparameters are correctly tweaked, we use the full dataset to fit
    y = df['Low'].values
    model = ARIMA(y, order=(5,0,1)).fit()
    forecast = model.forecast(steps=1)[0]

    # Returning the last real data and the forecast for the next minute
    return (y[len(y)-1], forecast)

In [121]:
df = get_finance_data('HFOF11.SA')

In [125]:
df['2020-01']

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-01-02,125.610001,130.000000,125.610001,126.150002,32453,0,0
2020-01-03,126.800003,127.000000,126.150002,126.790001,30646,0,0
2020-01-06,126.989998,127.000000,126.309998,127.000000,42594,0,0
2020-01-07,127.000000,127.930000,123.629997,127.000000,59164,0,0
2020-01-08,126.989998,126.989998,118.010002,123.800003,70903,0,0
2020-01-09,124.019997,126.889999,121.599998,123.550003,58318,0,0
2020-01-10,123.550003,124.949997,122.000000,122.400002,38604,0,0
2020-01-13,122.500000,123.930000,122.500000,123.190002,24324,0,0
2020-01-14,123.260002,125.000000,123.169998,124.500000,29911,0,0


In [87]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-12-23,123.470001,123.470001,122.000000,123.000000,2864,0,0
2019-12-23,123.000000,129.330002,122.800003,129.330002,12877,0,0
2019-12-23,126.720001,128.250000,123.029999,123.220001,8879,0,0
2019-12-23,123.220001,123.790001,122.699997,122.699997,4018,0,0
2019-12-23,122.690002,122.900002,122.010002,122.500000,3929,0,0
...,...,...,...,...,...,...,...
2020-12-22,102.919998,102.930000,102.750000,102.900002,3126,0,0
2020-12-22,102.900002,102.930000,102.849998,102.930000,4143,0,0
2020-12-22,102.930000,102.930000,102.750000,102.919998,2424,0,0


In [127]:
ativo = yf.Ticker('HFOF11.SA')

In [130]:
ativo.get_dividends()

Series([], Name: Dividends, dtype: int64)

In [ ]:
df = google.history(period=period, interval=interval)